In [1]:
import pandas as pd

competition_data = pd.read_csv('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/csv_files/competition_details.csv')
competition_data

,competition_id,competition_name,competition_parent_id,competition_type,competition_gender,category_id
0,sr:competition:620,Hopman Cup,NaN,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,NaN,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,NaN,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,NaN,mixed,men,sr:category:76
...,...,...,...,...,...,...
6034,sr:competition:46999,UTR Urbana W01,NaN,singles,women,sr:category:2517
6035,sr:competition:47045,"WTA 125K San Sebastian, Spain Women Singles",sr:competition:47043,singles,women,sr:category:871
6036,sr:competition:47047,"WTA 125K San Sebastian, Spain Women Doubles",sr:competition:47043,doubles,women,sr:category:871
6037,sr:competition:47053,"WTA 125K Caldas da Rainha, Portugal Women Singles",sr:competition:47051,singles,women,sr:category:871


save data in postgresql database

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


Creating DataBase:

In [3]:
pip install toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


Creating competition Table

In [5]:
competition_create_table_sql = """create table if not exists competition_table(
competition_id varchar(50) PRIMARY KEY, 
competition_name varchar(100) NOT NULL,
competition_parent_id varchar(50), 
competition_type varchar(20) NOT NULL, 
competition_gender varchar(10) NOT NULL,
categoryid varchar(50),
FOREIGN KEY(categoryid) references categories_table(categoryid))"""

#execute:
mycursor.execute(competition_create_table_sql)

print("Competition Table has been created")

Competition Table has been created


Fetching Column Name from categories_table:

In [6]:
column_name = """SELECT column_name FROM 
        information_schema.columns WHERE table_name = 'categories_table';"""
mycursor.execute(column_name)
fetching_column_names = mycursor.fetchall()
fetching_column_names

[('categoryid',), ('categoryname',)]

insert values into competition table:

In [7]:
competition_insert_table_sql = """Insert into competition_table(
competition_id, 
competition_name,
competition_parent_id,
competition_type,
competition_gender,
categoryid) 
values (%s, %s, %s, %s, %s, %s) 
ON CONFLICT (competition_id) DO UPDATE SET
    competition_name = EXCLUDED.competition_name,
    competition_parent_id = EXCLUDED.competition_parent_id,
    competition_type = EXCLUDED.competition_type,
    competition_gender = EXCLUDED.competition_gender,
    categoryid = EXCLUDED.categoryid;"""


#ON CONFLICT (competition_id) DO UPDATE SET
#If the row I'm trying to insert would violate a uniqueness constraint (like a duplicate primary key), then update the existing row, instead of throwing an error.


#competition_name = EXCLUDED.competition_name:-
# …it means: "update competition_name with the value I tried to insert."

In [8]:
for index, row in competition_data.iterrows():
    print(row) #We will get the o/p in the format of dictionary with column names as keys.

    print("Inserting:", tuple(row))  #The iter row (only values) will converts into python tuple 
    print("----------------------\n")
    mycursor.execute(competition_insert_table_sql, tuple(row)) #('sr:competition:620', 'Hopman Cup', nan, 'mixed', 'mixed', 'sr:category:181') this tuple of rows will be given to insert query values.
print("Data inserted  into the competition_table using iterrows()")


#.iterrows() allows you to loop through each row of the table.
     # index: The index of the row (e.g., 0, 1, 2...)
     # row: A Pandas Series containing the data for that row (column names are keys).

#tuple(row) turns that row into a Python tuple — needed by SQL

competition_id           sr:competition:620
competition_name                 Hopman Cup
competition_parent_id                   NaN
competition_type                      mixed
competition_gender                    mixed
category_id                 sr:category:181
Name: 0, dtype: object
Inserting: ('sr:competition:620', 'Hopman Cup', nan, 'mixed', 'mixed', 'sr:category:181')
----------------------

competition_id           sr:competition:660
competition_name             World Team Cup
competition_parent_id                   NaN
competition_type                      mixed
competition_gender                      men
category_id                   sr:category:3
Name: 1, dtype: object
Inserting: ('sr:competition:660', 'World Team Cup', nan, 'mixed', 'men', 'sr:category:3')
----------------------

competition_id                   sr:competition:990
competition_name         ATP Challenger Tour Finals
competition_parent_id           sr:competition:6239
competition_type                          

Fetching Data from competition_table:

In [9]:
#execute:
mycursor.execute("Select * from competition_table")

fetching_Data = mycursor.fetchall()
fetching_Data

[('sr:competition:620',
  'Hopman Cup',
  'NaN',
  'mixed',
  'mixed',
  'sr:category:181'),
 ('sr:competition:660',
  'World Team Cup',
  'NaN',
  'mixed',
  'men',
  'sr:category:3'),
 ('sr:competition:990',
  'ATP Challenger Tour Finals',
  'sr:competition:6239',
  'singles',
  'men',
  'sr:category:72'),
 ('sr:competition:1207',
  'Championship International Series',
  'NaN',
  'singles',
  'women',
  'sr:category:6'),
 ('sr:competition:2100', 'Davis Cup', 'NaN', 'mixed', 'men', 'sr:category:76'),
 ('sr:competition:2102',
  'Billie Jean King Cup',
  'NaN',
  'mixed',
  'women',
  'sr:category:74'),
 ('sr:competition:2555',
  'Wimbledon Men Singles',
  'sr:competition:2553',
  'singles',
  'men',
  'sr:category:3'),
 ('sr:competition:2557',
  'Wimbledon Men Doubles',
  'sr:competition:2553',
  'doubles',
  'men',
  'sr:category:3'),
 ('sr:competition:2559',
  'Wimbledon Women Singles',
  'sr:competition:2553',
  'singles',
  'women',
  'sr:category:6'),
 ('sr:competition:2561',
  'W

close cursor & db connection:

In [10]:
mycursor.close()
my_db_connection.close()